# Eigen funcX Flow


In [ ]:
import os
import sys
import time
import shortuuid

from funcx.sdk.client import FuncXClient
from fair_research_login import NativeClient
from globus_automate_client import create_flows_client
from globus_automate_client.token_management import CLIENT_ID

# Authenticate

Auth with the funcX and Automate clients.

Note: You will still need to grant access to the flow later on in this notebook.

In [ ]:
fxc = FuncXClient()

In [ ]:
flows_client = create_flows_client(CLIENT_ID)

In [ ]:
client = NativeClient(client_id='7414f0b4-7d05-4bb6-bb00-076fa3f17cf5')
tokens = client.login(requested_scopes=['https://auth.globus.org/scopes/56ceac29-e98a-440a-a594-b41e7a084b62/all'])
auth_token = tokens["petrel_https_server"]['access_token']
headers = {'Authorization': f'Bearer {auth_token}'}
print(headers)

# Test Setup
## Edit things here only!!

In [ ]:
# Container name
### should be the same within the container repo and resource
container_name = "eigen.simg"

In [ ]:
#funcx endpoint configuration for theta
theta_conf = {'endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
              'local_endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
              'data_dir': '/projects/APSDataAnalysis/Braid/data/XPCS/',
              'proc_dir': '/projects/APSDataAnalysis/Braid/process/',
              'cont_dir': '/home/rvescovi/.funcx/containers/'}

In [ ]:
#funcx endpoint configuration for midway 
midway_conf = {'endpoint': '159eeda6-d2c3-4e87-b9b6-98711a938b48',
              'local_endpoint': 'dede52af-cee9-4e44-b017-ceb0c8f584cb',
              'data_dir': '/home/ravescovi/workspace/Braid/data/XPCS/',
              'proc_dir': '/home/ravescovi/workspace/Braid/process/',
              'cont_dir': '/home/ravescovi/.funcx/containers/'}

In [ ]:
# Choose which resource to run.
conf = theta_conf

In [ ]:
# Register your container
### Even though we download the container during the flow.
#### We need to create a funcx id for it.
eigen_cont_id = fxc.register_container(location=os.path.join(conf['cont_dir'],container_name), container_type='singularity')

# funcX setup
Register the containers and functions for the flow.

In [ ]:
def download_cont(data):
    """Download the container and dataset"""
    import os
    import requests


    server_url = data.get('container_server_url', "")
    container_name = data.get('container_name', "")
    headers = data['headers']

    if server_url==None:
        raise(NameError('No container `server URL` specified'))
    if container_name==None:
        raise(NameError('No container `name` specified'))

    container_url = os.path.join(server_url,container_name)
    
    ##deal with container path in the system (need to be consistent with container_uid)
    container_path = data.get('container_path', '')
    if not container_path:
        os.path.join(os.path.expanduser("~"),'.funcx/containers')

    if not os.path.exists(data['container_path']):
        os.mkdir(data['container_path'])

    #deal with overwrite?
    if not os.path.isfile(data['container_path']):
        container_full_name = os.path.join(data['container_path'],data['container_name'])
        
        r = requests.get(container_url, headers=headers)
        if not r.status_code==200:  
            raise r.raise_for_status()
        open(container_full_name , 'wb').write(r.content)

    return container_full_name

download_cont_fxid = fxc.register_function(download_cont)

In [ ]:
def download_data(data):
    import os
    import requests

    data_dir = data['data_dir']
    headers = data['headers']
    dataset_server = data['dataset_server_url']
    dataset_name = data['dataset_name']

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    dataset_path = os.path.join(data_dir, dataset_name)
    dataset_url = os.path.join(dataset_server,dataset_name)

    if not os.path.isfile(dataset_path):
        r = requests.get(dataset_url, headers=headers)
        if not r.status_code==200:  
            raise r.raise_for_status()
        open(dataset_path , 'wb').write(r.content)
    
    return dataset_path

download_data_fxid = fxc.register_function(download_data)

In [ ]:
def unwrap_data(data):
    import os
    import tarfile


    data_dir = data['data_dir']
    dataset_name = data['dataset_name']

    dataset_path = os.path.join(data_dir, dataset_name)
    
    folder_name = dataset_name.split(".")[0] ##change for os.path.filename??
    folder_path = os.path.join(data_dir,folder_name)
    os.mkdir(folder_path)
    
    if os.path.isfile(dataset_path):
        with tarfile.open(dataset_path) as file:
            file.extractall(folder_path)
        return folder_path
    else:
        raise 'file does not exist!'
  
unwrap_data_fxid = fxc.register_function(unwrap_data)

In [ ]:
def xpcs_corr(event):
    import os
    import subprocess
    from subprocess import PIPE

    
    proc_dir = event['proc_dir']
    if not os.path.exists(proc_dir):
        os.mkdir(proc_dir)

    hdf_file = event['data']['hdf']
    imm_file = event['data']['imm']
    
    flags = event['data'].get('flags', "")

    cmd = f"corr {hdf_file} -imm {imm_file} {flags}"
  
    res = subprocess.run(cmd, stdout=PIPE, stderr=PIPE,
                             shell=True, executable='/bin/bash')
    
    with open(os.path.join(proc_dir,'corr_output.log'), 'w+') as f:
                f.write(res.stdout.decode('utf-8'))

    with open(os.path.join(proc_dir,'corr_errors.log'), 'w+') as f:
                f.write(res.stderr.decode('utf-8'))
    
    return str(res.stdout)

corr_fxid = fxc.register_function(xpcs_corr, container_uuid=eigen_cont_id)

# Flow Check Functions

In [ ]:
from IPython.display import clear_output
import json
from pprint import pprint
from datetime import datetime, timezone

def flow_check(flow_action, refresh=10):
    
    start = datetime.fromisoformat(flow_action['start_time'])

    while True:
        flow_action_id = flow_action['action_id']
        flow_action = flows_client.flow_action_status(flow_id, flow_scope, flow_action_id)
        flow_status = flow_action['status']

        print(f'Flow status: {flow_status}')
        
        if flow_status == 'ACTIVE': 
            now = datetime.now(timezone.utc)
            print(f'Time elapsed: {now - start}')
            pprint(json.dumps(flow_action.data, indent = 2, sort_keys=True))

        elif flow_status == 'FAILED':
            complete = datetime.fromisoformat(flow_action['completion_time'])
            print(f'Time elapsed: {complete - start}')
            break
        elif flow_status == 'SUCCEEDED':
            complete = datetime.fromisoformat(flow_action['completion_time'])
            print(f'Time elapsed: {complete - start}')
            break

        clear_output(wait=True)
        time.sleep(refresh)



# Create the flow

In [ ]:
flow_definition = {
  "Comment": "Eigen Flow",
  "StartAt": "Download Container",
  "States": {
    "Download Container": {
      "Comment": "Download the container",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/all",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.funcx_local_ep",
            "func.$": "$.input.download_cont_fxid",
            "payload.$": "$.input"
        }]
      },
      "ResultPath": "$.Exec1Result",
      "WaitTime": 600,
      "Next": "Download Data"
    },
    "Download Data": {
      "Comment": "Download the data",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/all",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.funcx_local_ep",
            "func.$": "$.input.download_data_fxid",
            "payload.$": "$.input"
        }]
      },
      "ResultPath": "$.Exec2Result",
      "WaitTime": 600,
      "Next": "Unwrap Data"
    },
    "Unwrap Data": {
      "Comment": "Unwrap the data",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/all",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.funcx_local_ep",
            "func.$": "$.input.download_data_fxid",
            "payload.$": "$.input"
        }]
      },
      "ResultPath": "$.Exec3Result",
      "WaitTime": 600,
      "Next": "Eigen Corr"
    },
    "Eigen Corr": {
      "Comment": "Eigen Corr",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/all",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.funcx_ep",
            "func.$": "$.input.corr_fxid",
            "payload.$": "$.input"
        }]
      },
      "ResultPath": "$.Exec4Result",
      "WaitTime": 600,
      "End": True
    }
  }
}

In [ ]:
flow = flows_client.deploy_flow(flow_definition, title="Eigen Example flow")
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']
print(f'Newly created flow with id:\n{flow_id}')

# Define input for the flow

The input to the flow needs to specify what data to process, where it is located, and where to put it for analysis. The flow also requires the funcX function endpoint ids to use.

In [ ]:
#Set the name for the processing folder intermediate results
experiment_name = 'braid_eigen'
run_name = experiment_name + '_' + shortuuid.uuid()

print(run_name)

In [ ]:
# Specify the range of files to process

flow_input = {
    "input": {
        #HTTPS-Download Container variables
        "container_server_url":"https://45a53408-c797-11e6-9c33-22000a1e3b52.e.globus.org/Braid/containers",
        "container_name": "eigen_example.simg",
        "container_path": conf['cont_dir'],
        "headers": headers,

        #SingleFile-Download Variables
        "dataset_server_url": "https://45a53408-c797-11e6-9c33-22000a1e3b52.e.globus.org/Braid/data/xpcs_example/",
        "dataset_name": 'A001_Aerogel.tar.xz',
        "data_dir": conf['data_dir'],

        #Processing variables
        "proc_dir": os.path.join(conf['proc_dir'],run_name),

        #Eigen specific variables
        "data": {
            "hdf": f"{conf['data_dir']}/A001_Aerogel_1mm_att6_Lq0_001_0001-1000/A001_Aerogel_1mm_att6_Lq0_001_0001-1000.hdf",
            "imm": f"{conf['data_dir']}/A001_Aerogel_1mm_att6_Lq0_001_0001-1000/                 A001_Aerogel_1mm_att6_Lq0_001_00001-01000.imm",
        },

        #Eigen funcX functions
        "corr_fxid": corr_fxid,

        #Utility funcX functions
        "download_cont_fxid": download_cont_fxid,
        "download_data_fxid": download_data_fxid,
        "unwrap_data_fxid": unwrap_data_fxid,

        # funcX endpoints 
        "funcx_ep": conf['endpoint'],
        "funcx_local_ep": conf['local_endpoint'],
    }
}

# Run the flow

This will require you to authenticate and grant access to the flow to use Transfer and funcX on your behalf.

The flow should take a couple of minutes.

In [ ]:
flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)


In [ ]:
flow_check(flow_action, refresh=1)